# 1. Data Loading and Exploration

This notebook demonstrates how to load and explore the multi-modal data from the Dareplane project. We will cover:

- Loading XDF files for a single session.
- Inspecting the `mne.Raw` object containing LFP, ECoG, and EOG data.
- Visualizing the raw neural data and event markers.
- Extracting and visualizing the stylus tracing coordinates from the behavioral data stream.

## 1.1 Setup

First, let's import the necessary libraries and add the `analysis_scripts` directory to our Python path to access the custom loading functions.

In [1]:
from pathlib import Path

import mne
import numpy as np
import pandas as pd
import pyxdf
from scipy.io import loadmat
from tqdm import tqdm
import os
import sys

sys.path.append(str(Path.cwd().parent.parent / "analysis_scripts"))

from load_xdf import create_raws_from_mat_and_xdf

## 1.2 Load Neural Data

Now, we'll load the data for a specific session. The `get_xdf_files` function helps us find the relevant XDF files, and `create_raws_from_mat_and_xdf` loads the data into a list of `mne.Raw` objects, one for each block in the session.

In [2]:
CHANNEL_MAP = {
    "CECOG_HF_1___01___Array_1___01": "LFP_1",
    "CECOG_HF_1___02___Array_1___02": "LFP_2",
    "CECOG_HF_1___03___Array_1___03": "LFP_3",
    "CECOG_HF_1___04___Array_1___04": "LFP_4",
    "CECOG_HF_1___05___Array_1___05": "LFP_5",
    "CECOG_HF_1___06___Array_1___06": "LFP_6",
    "CECOG_HF_1___07___Array_1___07": "LFP_7",
    "CECOG_HF_1___08___Array_1___08": "LFP_8",
    "CECOG_HF_1___09___Array_2___09": "LFP_9",
    "CECOG_HF_1___10___Array_2___10": "LFP_10",
    "CECOG_HF_1___11___Array_2___11": "LFP_11",
    "CECOG_HF_1___12___Array_2___12": "LFP_12",
    "CECOG_HF_1___13___Array_2___13": "LFP_13",
    "CECOG_HF_1___14___Array_2___14": "LFP_14",
    "CECOG_HF_1___15___Array_2___15": "LFP_15",
    "CECOG_HF_1___16___Array_2___16": "LFP_16",
    "CECOG_HF_2___01___Array_3___01": "ECOG_1",
    "CECOG_HF_2___02___Array_3___02": "ECOG_2",
    "CECOG_HF_2___03___Array_3___03": "ECOG_3",
    "CECOG_HF_2___04___Array_3___04": "ECOG_4",
    "CECOG_HF_2___05___Array_3___05": "EOG_1",
    "CECOG_HF_2___06___Array_3___06": "EOG_2",
    "CECOG_HF_2___07___Array_3___07": "EOG_3",
    "CECOG_HF_2___08___Array_3___08": "EOG_4",
}

In [3]:
ROOT_PATH = str(Path.cwd().parent.parent)
DATA_PATH = os.path.join(ROOT_PATH, "data")

In [4]:
def get_xdf_files(day: str = "day3", paradigms: str = "lsl") -> list:
    directory = f"{DATA_PATH}/sub-P001_ses-{day}/{paradigms}/"
    return [
        os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".xdf")
    ]

In [5]:
xdf_files = get_xdf_files(day="day3", paradigms="lsl")

In [7]:
def get_AO_file_data(fname: Path) -> dict:
    """Finds and loads the .mat file corresponding to an .xdf file."""
    stem = fname.stem
    pp = fname.parents[1].joinpath("AO")
    files = list(pp.glob(f"{stem}*mat"))
    if not files:
        raise FileNotFoundError(f"No .mat file found for {fname.name}")
    return {f: loadmat(f) for f in files}

In [8]:
def get_AO_data(fname, with_cport_marker: bool = True) -> pd.DataFrame:
    """
    Loads the high-quality, uninterrupted neural data from the Neuro Omega .mat file.
    This is the "ground truth" for the neural signal.
    """
    d = get_AO_file_data(fname)
    data = d[list(d.keys())[-1]]
    dm = data["CECOG_HF_2___01___Array_3___01"][0]
    tstart = data["CECOG_HF_2___01___Array_3___01_TimeBegin"][0]
    tm = np.linspace(0, int(len(dm) / 22_000), len(dm))
    df = pd.DataFrame({"time": tm, "data": dm, "src": "AO"})
    df = df.assign(**{v: data[k][0] for k, v in CHANNEL_MAP.items()})

    if with_cport_marker and "CPORT__1" in data:
        cport_ix = (
            ((data["CPORT__1"][0] / (data["CPORT__1_KHz"] * 1000) - tstart) * 22_000)
            .astype(int)
            .flatten()
        )
        marker = data["CPORT__1"][1]
        df.loc[cport_ix[marker != 0], "marker"] = marker[marker != 0]
    return df

In [9]:
def get_xdf_data(fname) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Loads the LSL data from the .xdf file, which contains the (potentially gappy)
    neural signal stream and the accurate experimental event markers.
    """
    xdf_data, _ = pyxdf.load_xdf(fname, dejitter_timestamps=False)
    streams = {d["info"]["name"][0]: d for d in xdf_data}

    ao_stream = streams["AODataStream"]
    marker_stream = streams["CopyDrawParadigmMarkerStream"]

    # Create DataFrame for the LSL neural data
    dx = ao_stream["time_series"][:, 16]  # Using ECOG_1 as reference signal
    tx = ao_stream["time_stamps"]
    df_lsl = pd.DataFrame({"time": tx, "data": dx, "src": "LSL"}).sort_values("time")

    # Add markers to the LSL DataFrame
    m_times = marker_stream["time_stamps"]
    m_series = marker_stream["time_series"][:, 0]

    # Align markers to the closest timestamp in the LSL neural data
    ix_marker = [np.argmin(np.abs(df_lsl.time - v)) for v in m_times]
    df_lsl.loc[ix_marker, "marker"] = m_series

    # Normalize time to start at 0
    tmin = df_lsl.time.iloc[0]
    df_lsl.time -= tmin

    # Create a separate, clean DataFrame for just the markers and their precise timestamps
    df_markers = pd.DataFrame({"time": m_times, "marker": m_series})
    df_markers.time -= tmin

    return df_lsl, df_markers

In [10]:
def align_on_markers(dfao: pd.DataFrame, dts: pd.DataFrame) -> pd.DataFrame:
    """
    Aligns AO and LSL data by matching the time intervals between hardware markers.
    This is the preferred method when reliable hardware markers exist in the .mat file.
    """
    dt_lsl = np.diff(dts[dts.marker.notna()].time)
    dt_ao = np.diff(dfao[dfao.marker.notna()].time)

    # Find a matching sequence of marker intervals
    ii = None
    for i in range(3):
        for j in range(3):
            if np.abs(dt_ao[i : i + j + 1].sum() - dt_lsl[0]) / dt_ao[0] < 0.1:
                ii = i if ii is None else ii
    if ii is None:
        raise KeyError("No matching marker distances found")

    # Transfer LSL markers to the AO timeline
    dists = dt_lsl.cumsum()
    t0idx = dfao[dfao.marker.notna()].index[ii]
    t0 = dfao.time.iloc[t0idx]
    idx = [t0idx] + [np.searchsorted(dfao.time, t0 + v) for v in dists]
    dfao["lsl_marker"] = np.nan
    dfao.loc[idx, "lsl_marker"] = dts[dts.marker.notna()].marker.values
    return dfao

In [11]:
def align_on_stim_artifact(
    dfao: pd.DataFrame, dflsl: pd.DataFrame, threshold: float = 2000
) -> pd.DataFrame:
    """
    Fallback alignment method. Aligns data by finding the first large stimulation
    artifact peak present in both the AO and LSL signals.
    """
    # Find the index of the first major peak (the stim artifact)
    ix_lsl_first_peak = dflsl[(dflsl.data > threshold) & (dflsl.time > 1)].index[0]
    ix_ao_first_peak = dfao[(dfao.data > threshold) & (dfao.time > 1)].index[0]

    # Calculate the time offset between the two streams
    time_offset = (
        dfao.iloc[ix_ao_first_peak]["time"] - dflsl.iloc[ix_lsl_first_peak]["time"]
    )

    # Fine-tune the offset with a small cross-correlation
    chk_idx_range = 10_000
    chk_len = 400
    idx_lsl_first_marker_idx = dflsl[dflsl.marker.notna()].index[0]
    t_lsl_first_marker = dflsl.iloc[idx_lsl_first_marker_idx].time
    ao_test_idx = np.searchsorted(dfao.time, t_lsl_first_marker + time_offset)
    dao_chk = dfao.iloc[ao_test_idx - chk_idx_range : ao_test_idx + chk_idx_range]
    dlsl_chk = dflsl[idx_lsl_first_marker_idx : idx_lsl_first_marker_idx + chk_len]
    differences = np.asarray(
        [
            dao_chk.iloc[i : i + chk_len].data.values - dlsl_chk.data.values
            for i in range(2 * chk_idx_range - chk_len)
        ]
    )
    idx_shift = np.argmin(np.abs(differences).mean(axis=1)) - chk_idx_range
    fine_tuned_offset = time_offset + idx_shift / 22_000

    # Transfer LSL markers to the AO timeline using the calculated offset
    dm = dflsl[dflsl.marker.notna()]
    idx = [np.searchsorted(dfao.time, v + fine_tuned_offset) for v in dm.time]
    dfao["lsl_marker"] = np.nan
    dfao.loc[idx, "lsl_marker"] = dflsl[dflsl.marker.notna()].marker.values
    return dfao

In [12]:
df_lsl, df_markers = get_xdf_data(xdf_files[0])
print(f"  - Loaded LSL data with {len(df_markers)} markers from .xdf")

  - Loaded LSL data with 36 markers from .xdf


In [13]:
path_object = Path(xdf_files[0])
try:
    df_ao = get_AO_data(path_object, with_cport_marker=True)
    print(
        f"  - Loaded complete neural signal from .mat with {df_ao.marker.notna().sum()} hardware markers."
    )
except FileNotFoundError as e:
    print(f"Error: {e}")
    exit()

Error: No .mat file found for block4_copydraw_on.xdf


In [ ]:
# 1b. Align the two data streams
print("\n1b. Aligning the two data streams...")
aligned_ao_data = None
try:
    # First, try to align using the hardware markers. This is most reliable.
    aligned_ao_data = align_on_markers(df_ao.copy(), df_markers)
    print("  - Success: Aligned data using hardware markers.")
except (KeyError, IndexError) as e:
    # If marker alignment fails (e.g., no markers in .mat file), fall back
    # to aligning on the large DBS stimulation artifact.
    print(
        f"  - Marker alignment failed ({e}). Falling back to stim artifact alignment."
    )
    # Reload AO data without expecting markers, just in case
    df_ao_no_marker = get_AO_data(file_to_process, with_cport_marker=False)
    aligned_ao_data = align_on_stim_artifact(df_ao_no_marker.copy(), df_lsl)
    print("  - Success: Aligned data using stimulation artifact.")


1b. Aligning the two data streams...


NameError: name 'df_ao' is not defined

: 

In [ ]:
# At this point, `aligned_ao_data` is a pandas DataFrame containing the
# complete, high-quality neural signal, now with a new 'lsl_marker'
# column that has the correct event timings.
print(f"\nAlignment complete. Result is a DataFrame with shape {aligned_ao_data.shape}")
print(
    f"Found {aligned_ao_data.lsl_marker.notna().sum()} LSL markers now aligned to the AO data."
)

In [ ]:
# --- Step 2: Create an MNE Raw Object ---
print("\n--- Step 2: Converting aligned data to an MNE Raw object ---")

# 2a. Define the metadata for the MNE object
ch_names = [c for c in aligned_ao_data.columns if c.startswith(("LFP", "ECOG", "EOG"))]
ch_types = (
    ["dbs"] * 16  # 16 LFP channels are considered 'dbs' type
    + ["ecog"] * 4  # 4 ECoG channels
    + ["eog"] * 4  # 4 EOG channels
)
sfreq = 22000  # The original sampling frequency of the Neuro Omega system
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
print(f"  - Created MNE Info object for {len(ch_names)} channels at {sfreq} Hz.")

In [ ]:
# 2b. Extract data and convert from microvolts to volts (MNE standard)
neural_data_uV = aligned_ao_data[ch_names].to_numpy().T
neural_data_V = neural_data_uV * 1e-6

In [ ]:
# 2c. Create the MNE RawArray object
raw = mne.io.RawArray(neural_data_V, info)
raw._filenames = [file_to_process.name]
print("  - Created MNE RawArray object.")

In [ ]:
# 2d. Create and attach the annotations from our aligned markers
marker_df = aligned_ao_data[aligned_ao_data.lsl_marker.notna()]
onsets = marker_df.index / sfreq  # Onset time in seconds
durations = np.zeros_like(onsets)  # Duration is 0 for point-like events
descriptions = marker_df.lsl_marker.astype(str)
annotations = mne.Annotations(
    onset=onsets, duration=durations, description=descriptions
)
raw.set_annotations(annotations)
print(f"  - Attached {len(annotations)} annotations to the Raw object.")

In [ ]:
# --- Step 3: Resample the Data ---
print("\n--- Step 3: Resampling data to 300 Hz ---")
resample_freq = 300
raw.resample(resample_freq)
print(f"Resampling complete. New sampling frequency: {raw.info['sfreq']} Hz")

In [ ]:
# --- Final Inspection ---
print("\n--- Data loading and initial preparation finished! ---")
print("You can now analyze the 'raw' object.")

print("\nInfo for the processed data block:")
print(raw.info)

print("\nAnnotations (event markers) for the block:")
print(raw.annotations)

In [ ]:
# Plot the data to visually inspect it.
print("\nPlotting the first 30s of ECoG data...")
ecog_channels = [ch for ch in raw.ch_names if "ECOG" in ch]
raw.copy().pick(ecog_channels).plot(
    duration=30, start=0, n_channels=len(ecog_channels), scalings="auto"
)
print("Plot window opened. Close it to continue.")

In [ ]:
raws = create_raws_from_mat_and_xdf(xdf_files, day="day3")

Processing /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block4_copydraw_on.xdf
Failed for /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block4_copydraw_on.xdf: 'str' object has no attribute 'stem'
Processing /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block3_copydraw_off.xdf
Failed for /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block3_copydraw_off.xdf: 'str' object has no attribute 'stem'
Processing /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block12_copydraw_on.xdf


found likely XDF file corruption (unpack requires a buffer of 8 bytes), scanning forward to next boundary chunk.
found likely XDF file corruption (unpack requires a buffer of 8 bytes), scanning forward to next boundary chunk.


Failed for /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block12_copydraw_on.xdf: 'str' object has no attribute 'stem'
Processing /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block6_copydraw_on.xdf
Failed for /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block6_copydraw_on.xdf: 'str' object has no attribute 'stem'
Processing /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block5_copydraw_off.xdf
Failed for /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block5_copydraw_off.xdf: 'str' object has no attribute 'stem'
Processing /home/bobby/repos/latent-neural-dynamics-modeling/data/sub-P001_ses-day3/lsl/block7_copydraw_off.xdf


KeyboardInterrupt: 

In [ ]:
# The 'day' parameter in create_raws_from_mat_and_xdf seems to be used for handling
# session-specific alignment issues. We'll use a placeholder value for now.
# Using 'day3' as per function's default logic

# For demonstration, let's concatenate all blocks into a single Raw object
raw = mne.concatenate_raws(raws)

Processing block4_copydraw_on.xdf
Failed for block4_copydraw_on.xdf: file block4_copydraw_on.xdf does not exist.
Processing block3_copydraw_off.xdf
Failed for block3_copydraw_off.xdf: file block3_copydraw_off.xdf does not exist.
Processing block12_copydraw_on.xdf
Failed for block12_copydraw_on.xdf: file block12_copydraw_on.xdf does not exist.
Processing block6_copydraw_on.xdf
Failed for block6_copydraw_on.xdf: file block6_copydraw_on.xdf does not exist.
Processing block5_copydraw_off.xdf
Failed for block5_copydraw_off.xdf: file block5_copydraw_off.xdf does not exist.
Processing block7_copydraw_off.xdf
Failed for block7_copydraw_off.xdf: file block7_copydraw_off.xdf does not exist.
Processing block11_copydraw_off.xdf
Failed for block11_copydraw_off.xdf: file block11_copydraw_off.xdf does not exist.
Processing block2_copydraw_on.xdf
Failed for block2_copydraw_on.xdf: file block2_copydraw_on.xdf does not exist.
Processing block8_copydraw_on.xdf
Failed for block8_copydraw_on.xdf: file bloc

AttributeError: 'str' object has no attribute 'stem'

## 1.3 Explore the `mne.Raw` Object

The loaded data is now in an `mne.Raw` object. MNE-Python provides a rich interface for exploring this data.

In [ ]:
print(raw.info)

In [ ]:
print(f"Channel names: {raw.ch_names}")
print(f"Number of channels: {len(raw.ch_names)}")
print(f"Sampling frequency: {raw.info['sfreq']} Hz")

## 1.4 Visualize Raw Data and Events

Let's plot the raw time series data. We can also visualize the event markers, which are stored as `Annotations` in the `mne.Raw` object.

In [ ]:
# Plot the raw data (it's interactive!)
# We'll select a subset of channels and a short duration for clarity
picks = mne.pick_channels(raw.ch_names, include=["LFP_1", "LFP_2", "ECOG_1", "ECOG_2"])
raw.plot(start=0, duration=10, picks=picks, n_channels=4, scalings="auto")

In [ ]:
# The event markers are stored in raw.annotations
print(raw.annotations)

## 1.5 Extract and Visualize Stylus Tracing Data

The stylus (tracing) coordinates are stored in a separate stream within the XDF file. We need to load the XDF file again with `pyxdf` to access all streams and find the one containing the stylus data. Based on the task (CopyDraw), we can look for a stream with a name related to drawing or mouse coordinates.

In [ ]:
# Let's use the first XDF file from the session for this example
xdf_file_path = xdf_files[0]
streams, header = pyxdf.load_xdf(xdf_file_path)

stylus_stream = None
for stream in streams:
    # Heuristic to find the stylus stream: it might be named 'Mouse' or similar,
    # and will likely have 2 or 3 channels (X, Y, maybe pressure).
    stream_name = stream["info"]["name"][0]
    if (
        "Mouse" in stream_name
        or "Stylus" in stream_name
        or "Coordinates" in stream_name
    ):
        stylus_stream = stream
        break

if stylus_stream:
    print(f"Found stylus stream: {stylus_stream['info']['name'][0]}")
    stylus_data = stylus_stream["time_series"]
    stylus_timestamps = stylus_stream["time_stamps"]

    # Create a pandas DataFrame for easier handling
    df_stylus = pd.DataFrame(stylus_data, columns=["x", "y"])  # Assuming 2D coordinates
    df_stylus["time"] = stylus_timestamps

    # Plot the stylus trace
    fig = px.line(df_stylus, x="x", y="y", title="Stylus Trace")
    fig.update_yaxes(autorange="reversed")  # Often, screen Y coordinates are inverted
    fig.show()
else:
    print("Could not automatically find a stylus/mouse stream in the XDF file.")
    print("Available streams:")
    for stream in streams:
        print(f"- {stream['info']['name'][0]}")